In [22]:
import subprocess
import boto3
import os
from google.cloud import storage
import shutil
import h5py


In [23]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name,timeout=None)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name,timeout=None)

def processRTC(s3key,gcskey):
    if os.path.exists('./temp/'):
        shutil.rmtree('./temp/')
    os.mkdir('./temp/')
    os.mkdir('./temp/deflate/')

    #Download CSLC file
    print('Downloading CSLC File')
    session = boto3.session.Session(profile_name='saml-pub')
    bucket = 'opera-pst-rs-pop1'
    filename = s3key.split('/')[-1]
    filepath = './temp/'+filename
    s3 = session.client('s3')
    s3.download_file(bucket, s3key, filepath)

    #Change compression to DEFLATE
    print('Converting amplitude to COG')
    outfile = './temp/deflate/'+filename
    f = h5py.File(filepath, 'r')
    pass_direction = f['science']['SENTINEL1']['identification']['orbit_pass_direction'][()]
    if pass_direction == b'Ascending':
        pass_d = 'A'
    elif pass_direction == b'Descending':
        pass_d = 'D'
    else:
        pass_d = 'N'
    cslc_h5_amp = f'DERIVED_SUBDATASET:AMPLITUDE:NETCDF:"{filepath}":/science/SENTINEL1/CSLC/grids/VV'
    gdal_cmd = f'gdal_translate -of COG {cslc_h5_amp} {outfile}'
    subprocess.run(gdal_cmd,shell=True,)

    #Upload to GCS bucket
    print('Uploading to GCS Bucket')
    gcsbucket = "opera-bucket-cslc"
    gcskey = gcskey.split('.tif')[0] + f'_{pass_d}.tif'
    upload_blob(gcsbucket,outfile,gcskey)


In [24]:
#s3key = 'products/CSLC_S1/OPERA_L2_CSLC-S1A_IW_T034-071045-IW1_VV_20181222T002623Z_v0.0_20230315T003618Z/OPERA_L2_CSLC-S1A_IW_T034-071045-IW1_VV_20181222T002623Z_v0.0_20230315T003618Z.h5'
#gcskey = 'products/test-run/OPERA_L2_CSLC-S1A_IW_T034-071045-IW1_VV_20181222T002623Z_v0.0_20230315T003618Z.h5'
#processRTC(s3key,gcskey)

In [25]:
session = boto3.session.Session(profile_name='saml-pub')
client = session.client('s3')
result = client.list_objects(Bucket='opera-pst-rs-pop1', Prefix='products/int_fwd_r2/2023-05-04_globalrun_2021-04-11_to_2021-04-22/CSLC_S1/', Delimiter='/')
keyList = []
for o in result.get('CommonPrefixes'):
    path = o.get('Prefix')
    #if path[-2] != 's':  #checks if end is not "static layers"
    fname = path.split('/')[-2]+'.h5'
    keyList.append(path+fname)

    # print('sub folder : ', o.get('Prefix'))
print(len(keyList))


999


In [27]:
for key in keyList[2:40]:
    fname = key.split('/')[-1]
    gcsKey = 'products/2023-05-04_globalrun_2021-04-11_to_2021-04-22/'+fname.split('.h5')[0]+'.tif'
    processRTC(key,gcsKey)

Converting amplitude to COG
Input file size is 18570, 4551
0...10...20...30...40...50...60...70...80...90...100 - done.
Uploading to GCS Bucket
Converting amplitude to COG
Input file size is 19842, 4653
0...10...20...30...40...50...60...70...80...90...100 - done.
Uploading to GCS Bucket
Converting amplitude to COG
Input file size is 20622, 4593
0...10...20...30...40...50...60...70...80...90...100 - done.
Uploading to GCS Bucket
Converting amplitude to COG
Input file size is 18570, 4548
0...10...20...30...40...50...60...70...80...90...100 - done.
Uploading to GCS Bucket
Converting amplitude to COG
Input file size is 19842, 4647
0...10...20...30...40...50...60...70...80...90...100 - done.
Uploading to GCS Bucket
Converting amplitude to COG
Input file size is 20622, 4590
0...10...20...30...40...50...60...70...80...90...100 - done.
Uploading to GCS Bucket
Converting amplitude to COG
Input file size is 18570, 4545
0...10...20...30...40...50...60...70...80...90...100 - done.
Uploading to GCS

KeyboardInterrupt: 